In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
coupon_red=pd.read_csv('/content/drive/MyDrive/coupon_redempt.csv')
demo=pd.read_csv('/content/drive/MyDrive/hh_demographic.csv')

In [22]:
coupon_red.head()
#coupon_red.count()

,household_key,day,coupon_upc,campaign
0,1,421,10000085364,8
1,1,421,51700010076,8
2,1,427,54200000033,8
3,1,597,10000085476,18
4,1,597,54200029176,18


In [5]:
demo.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [9]:
#Joining the dataframes 
demo.columns = map(str.lower, demo.columns)
coupon_red.columns = map(str.lower, coupon_red.columns)
hh_coupon=coupon_red.groupby('household_key')['coupon_upc'].count().reset_index()
hh_coupon.head()

,household_key,coupon_upc
0,1,5
1,8,1
2,13,21
3,14,3
4,18,8


In [23]:
hh_coupon.count()

household_key    434
coupon_upc       434
dtype: int64

In [17]:
#demo.join(hh_coupon,on=demo.household_key,lsuffix=)

demo.join(hh_coupon.set_index('household_key'),on='household_key',how='inner')

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,coupon_upc
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,5
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,1
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,21
6,45-54,A,100-124K,Homeowner,2 Adults No Kids,2,None/Unknown,18,8
7,35-44,B,15-24K,Unknown,Single Female,1,None/Unknown,19,1
...,...,...,...,...,...,...,...,...,...
790,35-44,A,75-99K,Homeowner,2 Adults Kids,3,1,2467,5
794,45-54,B,35-49K,Homeowner,Single Female,1,None/Unknown,2488,11
795,35-44,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2489,28
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494,5
